In [9]:
from mastodon import Mastodon

API = Mastodon(
    client_id='b-1LjDLQvdfa1Kd_uVk4EStyVGWCe_lqipdwKG68LGw',
    client_secret='jkWM5f58nofXVQP9-x0sf6yzuB3WfePTiIaHLe97kwM',
    access_token='zuG68yri_Fp4RxFro2QzRK4RRAp1bcmFwyRc5Rjb9Sw',
    api_base_url='https://mastodon.social'  
)

me = API.account_verify_credentials()
print(me)

Account([('id', 115469357557448283), ('username', 'vuTester'), ('acct', 'vuTester'), ('display_name', 'bendover'), ('discoverable', True), ('group', False), ('locked', False), ('created_at', datetime.datetime(2025, 10, 31, 0, 0, tzinfo=tzutc())), ('following_count', 0), ('followers_count', 0), ('statuses_count', 0), ('note', '<p>bendover</p>'), ('url', 'https://mastodon.social/@vuTester'), ('uri', 'https://mastodon.social/ap/users/115469357557448283'), ('avatar', 'https://files.mastodon.social/accounts/avatars/115/469/357/557/448/283/original/a9e38e3d0d41d484.png'), ('header', 'https://files.mastodon.social/accounts/headers/115/469/357/557/448/283/original/f859c2745152ea94.png'), ('avatar_static', 'https://files.mastodon.social/accounts/avatars/115/469/357/557/448/283/original/a9e38e3d0d41d484.png'), ('header_static', 'https://files.mastodon.social/accounts/headers/115/469/357/557/448/283/original/f859c2745152ea94.png'), ('moved', None), ('suspended', None), ('limited', None), ('bot', 

In [12]:
import time
from datetime import datetime, timedelta, timezone

events = [
    {
        "name": "ChatGPT launch",
        "terms": ["chatgpt", "gpt-4"],
        "event_date": datetime(2022, 11, 30, tzinfo=timezone.utc),
    },
    {
        "name": "Stable Diffusion release",
        "terms": ["stablediffusion", "midjourney"],
        "event_date": datetime(2022, 8, 22, tzinfo=timezone.utc),
    },
]

def to_datetime(dt):
    if isinstance(dt, datetime):
        return dt
    if dt.endswith('Z'):
        dt = dt[:-1] + '+00:00'
    return datetime.fromisoformat(dt)

def count_statuses_for_term_in_window(term, start_dt, end_dt, sleep_sec=1.0, max_loops=50):
    total = 0
    max_id = None
    loops = 0

    while True:
        res = API.search(q=term, result_type="statuses", max_id=max_id)
        statuses = res.get("statuses", [])
        if not statuses:
            break

        created_times = []
        for s in statuses:
            created_at = to_datetime(s["created_at"])
            created_times.append(created_at)
            if start_dt <= created_at < end_dt:
                total += 1

        max_id = min(int(s["id"]) for s in statuses)
        if min(created_times) < start_dt:
            break

        loops += 1
        if loops >= max_loops:
            break

        time.sleep(sleep_sec)

    return total

results = []

for event in events:
    name = event["name"]
    terms = event["terms"]
    event_date = event["event_date"]

    before_start, before_end = event_date - timedelta(days=7), event_date
    after_start, after_end = event_date, event_date + timedelta(days=7)

    print(f"\n=== Event: {name} ===")
    event_result = {"name": name, "before": {}, "after": {}}

    for term in terms:
        print(f"\nTerm: '{term}'")
        before_count = count_statuses_for_term_in_window(term, before_start, before_end)
        print(f"  Before: {before_count}")
        after_count = count_statuses_for_term_in_window(term, after_start, after_end)
        print(f"  After:  {after_count}")

        event_result["before"][term] = before_count
        event_result["after"][term] = after_count

    results.append(event_result)


=== Event: ChatGPT launch ===

Term: 'chatgpt'
  Before: 0
  After:  0

Term: 'gpt-4'
  Before: 0
  After:  0

=== Event: Stable Diffusion release ===

Term: 'stablediffusion'
  Before: 0
  After:  0

Term: 'midjourney'


KeyboardInterrupt: 

In [ ]:
import requests
import time
import re
from collections import Counter
from datetime import datetime, timezone
from bs4 import BeautifulSoup

instances = [
    "sigmoid.social",
    "mastodon.ai",
    "ai.community",
    "social.ai",
    "mstdn.art",
    "creators.social",
    "glitch.social",
    "fediscience.org",
    "datasci.social",
    "scicomm.xyz",
    "econtwitter.net",
    "mastodon.social",
    "fosstodon.org",
    "techhub.social",
]

START_DATE = datetime(2022, 1, 1, tzinfo=timezone.utc)

stopwords = {
    "the", "and", "for", "that", "this", "you", "with", "are", "was", "have",
    "but", "not", "from", "your", "just", "like", "they", "their", "about",
    "what", "when", "where", "there", "them", "out", "get", "can", "all",
    "one", "will", "into", "more", "also", "some", "has", "had", "our",
    "any", "who", "why", "how", "its", "she", "him", "her", "then", "than",
    "too", "very", "did", "does", "didn", "don", "could", "would", "should",
}

def parse_dt(dt_str):
    if isinstance(dt_str, datetime):
        return dt_str
    if dt_str.endswith("Z"):
        dt_str = dt_str[:-1] + "+00:00"
    return datetime.fromisoformat(dt_str)

def fetch_instance_statuses(instance, start_date, max_pages=300, limit=40, sleep=1.0):
    base = f"https://{instance}/api/v1/timelines/public"
    max_id = None
    statuses = []

    for _ in range(max_pages):
        params = {"local": "true", "limit": limit}
        if max_id is not None:
            params["max_id"] = max_id

        r = requests.get(base, params=params, timeout=30)
        r.raise_for_status()
        batch = r.json()
        if not batch:
            break

        stop = False
        for s in batch:
            created_at = parse_dt(s["created_at"])
            if created_at < start_date:
                stop = True
                break
            statuses.append(s)

        max_id = min(int(s["id"]) for s in batch)

        if stop:
            break

        time.sleep(sleep)

    return statuses

def extract_words(text):
    text = text.lower()
    text = re.sub(r"https?://\S+", " ", text)
    return re.findall(r"\b\w+\b", text)

word_freqs = {}
hashtag_freqs = {}

for inst in instances:
    print(f"\nFetching from {inst}...")
    statuses = fetch_instance_statuses(inst, START_DATE, max_pages=200, limit=40, sleep=1.0)
    print(f"  Got {len(statuses)} statuses")

    wc = Counter()
    hc = Counter()

    for s in statuses:
        html = s.get("content", "")
        text = BeautifulSoup(html, "html.parser").get_text(" ")
        words = [
            w for w in extract_words(text)
            if w not in stopwords and len(w) > 2
        ]
        wc.update(words)

        tags = s.get("tags", [])
        hc.update(t["name"].lower() for t in tags if "name" in t)

    word_freqs[inst] = wc
    hashtag_freqs[inst] = hc

TOP_N = 30

for inst in instances:
    print(f"\n==================== {inst} ====================")
    print("Top words:")
    for word, count in word_freqs[inst].most_common(TOP_N):
        print(f"{word:20} {count}")

    print("\nTop hashtags:")
    for tag, count in hashtag_freqs[inst].most_common(TOP_N):
        print(f"#{tag:19} {count}")